In [53]:
import sys
sys.path.append('../Module')

from loader_csv import charger_csv
from display_value import display_unique_categorical_values
from utils import generate_description, outliers


# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)

/tmp/ipykernel_3707/318031324.py:16: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df = charger_csv(partie1, partie2, partie3, partie4)


In [54]:
df.shape

(899164, 27)

In [55]:
# Utiliser la fonction pour générer la description de votre DataFrame
generate_description(df)


,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[8723173004, 1361854003, 1128474000, 481066300..."
1,Name,object,14,0.00,779583,"[FJH CONSULTING INC, SCOLARO TOOL COMPANY, BAS..."
2,City,object,30,0.00,32581,"[WISCONSIN DELLS, LEITCHFIELD, SOUTH BEND, Mer..."
3,State,object,14,0.00,51,"[FL, MI, IA, WI, NH]"
4,Zip,int64,0,0.00,33611,"[1603, 3110, 15217, 20019, 98273]"
5,Bank,object,1559,0.17,5802,"[BANK OF AMERICA NATL ASSOC, PNC BANK, NATIONA..."
6,BankState,object,1566,0.17,56,"[NC, NY, TX, SC, CA]"
7,NAICS,int64,0,0.00,1312,"[235310, 621310, 0, 722110, 812320]"
8,ApprovalDate,object,0,0.00,9859,"[24-Nov-03, 6-Dec-02, 18-Apr-03, 17-Jan-03, 12..."
9,ApprovalFY,object,0,0.00,71,"[2006, 1997, 2003, 1996, 1998]"


In [57]:
percentage = outliers(df['NoEmp'])
print(f'Percentage of outliers: {percentage:.2f}%')

TypeError: outliers() missing 1 required positional argument: 'series'

In [ ]:
df[df['NAICS'] ==0]

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"
6,1000093009,MIDDLE ATLANTIC SPORTS CO INC,UNION,NJ,7083,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,...,N,N,24-Jun-91,22-Jul-80,"$600,000.00",$0.00,CHGOFF,"$208,959.00","$600,000.00","$499,998.00"
9,1000124001,INTEXT BUILDING SYS LLC,GLASTONBURY,CT,6073,WEBSTER BANK NATL ASSOC,CT,0,28-Feb-97,1997,...,N,Y,NaN,30-Apr-97,"$70,000.00",$0.00,P I F,$0.00,"$70,000.00","$56,000.00"
13,1000154010,"ORCHARD CAFE & BAKERY, INC.",SLATERSVILLE,RI,2876,CITIZENS BANK NATL ASSOC,RI,0,28-Feb-97,1997,...,N,N,NaN,31-May-97,"$370,000.00",$0.00,P I F,$0.00,"$370,000.00","$277,500.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899153,9995413002,NORTH SHORE FLORAL,WOODBURY,NY,11797,FLUSHING BANK,NY,0,27-Feb-97,1997,...,0,N,7-Jul-04,31-May-97,"$142,000.00",$0.00,CHGOFF,$0.00,"$142,000.00","$106,500.00"
899154,9995423005,"LITWIN LIVERY SERVICES, INC.",CAMPBELL,OH,44405,JPMORGAN CHASE BANK NATL ASSOC,IL,0,27-Feb-97,1997,...,0,N,NaN,30-Sep-97,"$10,000.00",$0.00,P I F,$0.00,"$10,000.00","$5,000.00"
899158,9995563001,SHADES WINDOW TINTING AUTO ALA,IRVING,TX,75062,LOANS FROM OLD CLOSED LENDERS,DC,0,27-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$79,000.00",$0.00,P I F,$0.00,"$79,000.00","$63,200.00"
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27-Feb-97,1997,...,N,Y,8-Mar-00,31-Mar-97,"$75,000.00",$0.00,CHGOFF,"$46,383.00","$75,000.00","$60,000.00"


In [ ]:
df.NAICS
NAICSTrue = 0
NAICSFalse = 0
for i in df.NAICS.astype(str):
    if len(i) == 6:
        NAICSTrue += 1
    else:
        NAICSFalse += 1
print(NAICSTrue)
print(NAICSFalse)

697216
201948


In [ ]:
df = df[df.NAICS != 0]
df.shape

(697216, 27)

In [ ]:
# Filter rows based on conditions
missing_mis_status = df[(df['MIS_Status'].isna() & df['ChgOffDate'].notna()) | df['MIS_Status'].isna() & (df['ChgOffPrinGr'] != 0.0)]

# Print relevant columns
print(missing_mis_status[['MIS_Status', 'ChgOffDate', 'ChgOffPrinGr']])

# Update 'MIS_Status' for filtered rows
df.loc[missing_mis_status.index, 'MIS_Status'] = 'CHGOFF'

       MIS_Status ChgOffDate ChgOffPrinGr
833           NaN        NaN       $0.00 
897           NaN        NaN       $0.00 
1051          NaN        NaN       $0.00 
1115          NaN        NaN       $0.00 
3005          NaN        NaN       $0.00 
...           ...        ...          ...
875154        NaN        NaN       $0.00 
875227        NaN        NaN       $0.00 
875441        NaN        NaN       $0.00 
885904        NaN        NaN       $0.00 
898182        NaN        NaN       $0.00 

[1716 rows x 3 columns]


In [ ]:
print(df.loc[missing_mis_status.index, 'MIS_Status'])

833       CHGOFF
897       CHGOFF
1051      CHGOFF
1115      CHGOFF
3005      CHGOFF
           ...  
875154    CHGOFF
875227    CHGOFF
875441    CHGOFF
885904    CHGOFF
898182    CHGOFF
Name: MIS_Status, Length: 1716, dtype: object


In [ ]:
missing_bank_s = df[(df['BankState'].isna() & df['Bank'].isna())]

print(missing_bank_s.columns)
print(missing_bank_s[['BankState', 'Bank', 'City', 'State', 'MIS_Status']])

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')
       BankState Bank            City State MIS_Status
16108        NaN  NaN          DALLAS    TX      P I F
33663        NaN  NaN   RAQUETTE LAKE    NY      P I F
34582        NaN  NaN    OLMSTEDVILLE    NY      P I F
39732        NaN  NaN    OLMSTEDVILLE    NY      P I F
44475        NaN  NaN     WEST COVINA    CA      P I F
...          ...  ...             ...   ...        ...
866442       NaN  NaN          LOXLEY    AL      P I F
866444       NaN  NaN      FORT WORTH    TX      P I F
876947       NaN  NaN   MECHANICSBURG    PA      P I F
877019       NaN  NaN  MARINA DEL REY    CA      

In [ ]:
# Compter les occurrences de chaque catégorie dans la colonne 'MIS_Status'
mis_status_counts = missing_bank_s['MIS_Status'].value_counts()
print(mis_status_counts)


P I F     614
CHGOFF      8
Name: MIS_Status, dtype: int64


In [ ]:
df = df.drop(missing_bank_s.index, axis=0)
df.shape

(696594, 27)

In [ ]:
# Remplacer les valeurs manquantes dans 'City' par la valeur la plus fréquente pour cet État
df['City'] = df.groupby('State')['City'].transform(lambda x: x.fillna(x.mode()[0]))

# Remplacer les valeurs manquantes dans 'State' par la valeur la plus fréquente pour cette ville
df['State'] = df.groupby('City')['State'].transform(lambda x: x.fillna(x.mode()[0]))

# Supprimer les lignes avec des valeurs manquantes dans 'City' et 'State'
df.dropna(subset=['City', 'State'], inplace=True)

# Afficher les dimensions du DataFrame après le traitement
print("Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State':", len(df))

Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State': 696586


In [ ]:
# Sélectionner les observations avec BankState manquant et Bank non manquant
missing_Bank_State = df[(df['BankState'].isna() & df['Bank'].notna())]

# Afficher les valeurs de BankState et Bank pour les observations concernées
print(missing_Bank_State[['BankState', 'Bank']])

# Modifier les valeurs de la colonne BankState à 'HS'
df.loc[missing_Bank_State.index, 'BankState'] = 'HS'

       BankState                            Bank
145106       NaN             KOREA EXCHANGE BANK
424311       NaN  STANDARD CHARTERED BK KOREA LT
859233       NaN  THE BANK OF EAST ASIA, LIMITED


In [ ]:
# Afficher les valeurs de BankState et Bank pour les observations concernées après la modification
print(df.loc[missing_Bank_State.index, ['BankState', 'Bank']])

       BankState                            Bank
145106        HS             KOREA EXCHANGE BANK
424311        HS  STANDARD CHARTERED BK KOREA LT
859233        HS  THE BANK OF EAST ASIA, LIMITED


In [ ]:
df['RevLineCr'].value_counts()

N    270149
0    214208
Y    195196
T     14655
1        15
R        14
`         5
2         5
C         2
,         1
7         1
A         1
-         1
Q         1
Name: RevLineCr, dtype: int64

In [ ]:
mapping = {'0': 'N', '1': 'Y'}

# Remplacer les valeurs selon le mapping défini
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

# Supprimer les valeurs qui ne sont pas 'N', 'Y', ou 'T'
valid_values = ['N', 'Y', 'T']
df = df[df['RevLineCr'].isin(valid_values)]

# Afficher les nouvelles valeurs uniques pour vérification
print(df['RevLineCr'].value_counts())

N    484357
Y    195211
T     14655
Name: RevLineCr, dtype: int64


In [ ]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
maybe_leaking_feature = ['SBA_Appv', 'GrAppv', 'RetainedJob', 'CreateJob']
useless_feature = ['Name', 'Zip']
target = ['MIS_status']
df = df.drop(leaking_feature, axis=1)
df = df.drop(maybe_leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.shape

(694223, 16)

In [ ]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [LoanNr_ChkDgt, City, State, Bank, BankState, NAICS, ApprovalDate, ApprovalFY, Term, NoEmp, NewExist, FranchiseCode, UrbanRural, RevLineCr, LowDoc, MIS_Status]
Index: []


In [ ]:
# Utiliser la fonction pour afficher les valeurs uniques de chaque colonne
display_unique_categorical_values(df)

Colonne 'City':
- LOS ANGELES: 9723 (1.40%)
- HOUSTON: 7699 (1.11%)
- NEW YORK: 6671 (0.96%)
- MIAMI: 4908 (0.71%)
- CHICAGO: 4893 (0.70%)
- SAN DIEGO: 3940 (0.57%)
- DALLAS: 3723 (0.54%)
- PHOENIX: 3484 (0.50%)
- LAS VEGAS: 3417 (0.49%)
- BROOKLYN: 3387 (0.49%)
- DENVER: 2736 (0.39%)
- SALT LAKE CITY: 2686 (0.39%)
- PHILADELPHIA: 2623 (0.38%)
- SAN FRANCISCO: 2549 (0.37%)
- PORTLAND: 2539 (0.37%)
- AUSTIN: 2536 (0.37%)
- SEATTLE: 2526 (0.36%)
- SAN ANTONIO: 2508 (0.36%)
- SPRINGFIELD: 2480 (0.36%)
- ATLANTA: 2361 (0.34%)
- INDIANAPOLIS: 2319 (0.33%)
- ROCHESTER: 2228 (0.32%)
- COLUMBUS: 2129 (0.31%)
- PITTSBURGH: 2115 (0.30%)
- EL PASO: 2061 (0.30%)
- SAN JOSE: 1939 (0.28%)
- ORLANDO: 1884 (0.27%)
- MINNEAPOLIS: 1881 (0.27%)
- SACRAMENTO: 1864 (0.27%)
- LOUISVILLE: 1821 (0.26%)
- JACKSONVILLE: 1675 (0.24%)
- WASHINGTON: 1652 (0.24%)
- GLENDALE: 1634 (0.24%)
- OMAHA: 1584 (0.23%)
- ARLINGTON: 1559 (0.22%)
- CHARLOTTE: 1558 (0.22%)
- FORT WORTH: 1549 (0.22%)
- CINCINNATI: 1483 (0.21%)
-

In [ ]:
df.isna().sum()

LoanNr_ChkDgt       0
City                0
State               0
Bank                0
BankState           0
NAICS               0
ApprovalDate        0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          136
FranchiseCode       0
UrbanRural          0
RevLineCr           0
LowDoc           2582
MIS_Status          0
dtype: int64

In [ ]:
df.isna().sum()/df.count()

LoanNr_ChkDgt    0.000000
City             0.000000
State            0.000000
Bank             0.000000
BankState        0.000000
NAICS            0.000000
ApprovalDate     0.000000
ApprovalFY       0.000000
Term             0.000000
NoEmp            0.000000
NewExist         0.000196
FranchiseCode    0.000000
UrbanRural       0.000000
RevLineCr        0.000000
LowDoc           0.003733
MIS_Status       0.000000
dtype: float64

In [ ]:
df.head()

,LoanNr_ChkDgt,City,State,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status
0,1000014003,EVANSVILLE,IN,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,1,0,N,Y,P I F
1,1000024006,NEW PARIS,IN,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,2,2.0,1,0,N,Y,P I F
2,1000034009,BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,7,1.0,1,0,N,N,P I F
5,1000084002,PLAINVILLE,CT,"TD BANK, NATIONAL ASSOCIATION",DE,332721,28-Feb-97,1997,120,19,1.0,1,0,N,N,P I F
7,1000094005,SUMMERFIELD,FL,REGIONS BANK,AL,811118,28-Feb-97,1997,84,1,2.0,1,0,N,Y,P I F
